In [1]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

dataset_url = "../input/images-in-folder/images_in_folders"


# data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = "../input/images-in-folder/images_in_folders"
data_dir = "../input/images-in-folders-top10/images_in_folders - Copy"
# data_dir = "../input/animals/animals"
data_dir = pathlib.Path(data_dir)

img_height,img_width=224,224
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(classnames[labels[i]])
        plt.axis("off")

In [2]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

In [3]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(10, activation='softmax'))

In [4]:
# def recall(y_true, y_pred):
#     """Recall metric.

#     Only computes a batch-wise average of recall.

#     Computes the recall, a metric for multi-label classification of
#     how many relevant items are selected.
#     """
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall


resnet_model.summary()
resnet_model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [5]:
losses = []
learning_rates = []
class custom_callback(tf.keras.callbacks.Callback):
#     def on_epoch_begin(self, epoch, logs=None):
#         keys = list(logs.keys())
#         print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))
        if len(keys) > 2:
            print("loss is: ", logs[keys[2]])
            losses.append(round(logs[keys[2]], 2))
        
def scheduler(epoch, lr):
    if epoch < 4:
        print("latest losses are: ", losses)
        learning_rates.append(lr)
        return lr
    elif (losses[-2] == losses[-1]) and (losses[-3] == losses[-2]):
#             return lr * tf.math.exp(-0.1)
        print("latest losses are: ", losses[-3:])
        learning_rates.append(lr*0.9)
        return lr * 0.9
    learning_rates.append(lr)
    return lr

callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [59]:
from tensorflow.python.keras.callbacks import Callback
from sklearn.datasets import make_classification

class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y if (y.ndim == 1 or y.shape[1] == 1) else np.argmax(y, axis=1)
        self.reports = []

    def on_epoch_end(self, epoch, logs={}):
        y_hat = np.asarray(self.model.predict(self.x))
        y_hat = np.where(y_hat > 0.5, 1, 0) if (y_hat.ndim == 1 or y_hat.shape[1] == 1)  else np.argmax(y_hat, axis=1)
        report = classification_report(self.y,y_hat,output_dict=True)
        self.reports.append(report)
        return
   
    # Utility method
    def get(self, metrics, of_class):
        return [report[str(of_class)][metrics] for report in self.reports]


x, y = make_classification(n_features=6, n_redundant=0, n_informative=5,
                           random_state=1, n_clusters_per_class=1, n_classes=29)
# y = tf.keras.utils.to_categorical(y,29)
# metrics_multiclass = Metrics(x,y)
train_np = np.stack(list(train_ds))
print(train_np)

In [6]:
history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[callback, custom_callback()])

In [7]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.axis(ymin=0,ymax=0.5)

plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train_accuracy', 'val_accuracy'])

plt.show()

In [10]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train_accuracy', 'val_accuracy'])

plt.show()

In [8]:
fig1 = plt.gcf()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.axis(ymin=0,ymax=5)

plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')

plt.legend(['train_loss', 'val_loss'])

plt.show()

In [9]:
fig1 = plt.gcf()

plt.plot(learning_rates)

# plt.axis(ymin=0,ymax=0.01)

plt.grid()
plt.title('Model Learning Rate')
plt.ylabel('lr')
plt.xlabel('Epochs')


plt.show()

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# evaluate the model
score = resnet_model.evaluate(val_ds)

In [ ]:
score

In [ ]:
import pandas as pd

Books = pd.read_csv('../input/allbooks/allBooks.csv', encoding='cp1252')
Books

In [ ]:
import matplotlib.pyplot as plt
def show_plot(feature):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    genre_counts = Books[feature].value_counts()
    ax.bar(genre_counts.index,genre_counts)
    plt.xticks(rotation=-90)
    plt.show()
show_plot('Genre')

In [ ]:
show_plot('PublishYear')

In [ ]:
show_plot('Rate')

In [ ]:
import glob
from keras.preprocessing.image import image
image_list = {'BookId':[], 'image':[]}

for filename in glob.glob('../input/imagez/images/*.jpg'):
    image_list['BookId'].append(int(filename.split('/')[4].split('.')[0]))
    image_list['image'].append(filename)

images_df = pd.DataFrame(image_list)
images_df

In [ ]:
import os
main_df = pd.merge(Books, images_df, how='inner', on = 'BookId')

In [ ]:
# history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=10)

check_point = tf.keras.callbacks.ModelCheckpoint(filepath="cifar10.h5",
                                          monitor="val_acc",
                                          mode="max",
                                          save_best_only=True,
                                          )

y_train_tensor = tf.convert_to_tensor(y_train.to_numpy())
y_test_tensor = tf.convert_to_tensor(y_test.to_numpy())
X_train_tensor = tf.convert_to_tensor(X_train.to_numpy())
X_test_tensor = tf.convert_to_tensor(X_test.to_numpy())

history = resnet_model.fit(X_train_tensor, y_train_tensor, batch_size=32, epochs=10, verbose=1,
                    validation_data=(X_test_tensor, y_test_tensor),
                    callbacks=[check_point])

In [ ]:
import numpy as np
check_point = tf.keras.callbacks.ModelCheckpoint(filepath="cifar10.h5",
                                          monitor="val_acc",
                                          mode="max",
                                          save_best_only=True,
                                          )

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])
y_train_tensor = tf.convert_to_tensor(y_train.to_numpy())
y_test_tensor = tf.convert_to_tensor(y_test.to_numpy())
X_train_tensor = tf.convert_to_tensor(X_train.to_numpy())
X_test_tensor = tf.convert_to_tensor(X_test.to_numpy())
history = model.fit(X_train_tensor, y_train_tensor, batch_size=32, epochs=10, verbose=1,
                    validation_data=(X_test_tensor, y_test_tensor),
                    callbacks=[check_point])
# model.summary()

In [ ]:
# preprocess_data = __import__('0-transfer').preprocess_data

# fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase

_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
model = K.models.load_model('cifar10.h5')
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'ro', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
# # from PIL import Image
# from keras.preprocessing.image import image
# image = Image.open(main_df['image'][0])
# image.show()